In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Cargar los datos
df = pd.read_csv('candidatos_prueba.csv')

# Crear la variable objetivo
df['target'] = df['admitido']

# Eliminar las columnas no necesarias
df = df.drop(columns=['admitido', 'rechazado', 'en_proceso'])

# Separar las características (X) y la variable objetivo (y)
X = df.drop(columns=['target'])
y = df['target']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Definir el pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Definir los parámetros para GridSearchCV
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [10, 20, None],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy', verbose=2)

# Entrenar el modelo
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_model = grid_search.best_estimator_
print("Mejores hiperparámetros:", grid_search.best_params_)

# Predecir en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular el accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

# Guardar el modelo en un archivo pickle
model_filename = 'models/model_web.pkl'
joblib.dump(best_model, model_filename)
print(f"Modelo guardado como {model_filename}")


Fitting 5 folds for each of 54 candidates, totalling 270 fits
Mejores hiperparámetros: {'classifier__max_depth': 10, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 200}
Accuracy: 1.0000
Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       106
           1       1.00      1.00      1.00        94

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

Modelo guardado como models/model_web.pkl
